In [3]:
import os
from crewai import Agent, Task, Crew

In [4]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
)

In [5]:
os.environ["NVIDIA_NIM_API_KEY"] = "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"

In [ ]:
project_url = "https://github.com/crewai-ai/crewai"

In [4]:
# Create agents with different roles
code_analyzer = Agent(
  role='Code Analyzer',
  goal='Analyze code and documentation alignment',
  backstory="""You are an expert software engineer with years of experience in code analysis
  and documentation. Your strength lies in understanding code implementation and verifying if
  the documentation accurately reflects the codebase. You carefully review both the code and
  documentation to ensure they are in sync.""",
  verbose=True,
  llm="nvidia_nim/meta/llama-3.3-70b-instruct",
  tools=[
    DirectoryReadTool(directory="/Users/joaomoura/workspace/nvidia/crewAI/src/crewai"),
    FileReadTool()
  ]
)

documentation_writer = Agent(
  role='Documentation Writer',
  goal='Update and improve documentation based on code analysis',
  backstory="""You are a technical writer specialized in maintaining accurate documentation.
  You excel at understanding code analysis feedback and updating documentation to properly
  reflect the current implementation. You ensure documentation stays synchronized with
  the codebase while remaining clear and user-friendly.""",
  verbose=True,
  llm="nvidia_nim/meta/llama-3.3-70b-instruct",
  tools=[
    DirectoryReadTool(directory="/Users/joaomoura/workspace/nvidia/crewAI"),
    FileReadTool()
  ]
)

In [5]:
# Create tasks for the documentation process
analyze_code = Task(
    description="""Analyze the provided documentation file content:

    1. Read and understand the current documentation.
    2. Identify key concepts and components described.
    3. Compare documentation against actual codebase.
    4. Note any discrepancies or outdated information.
    5. Identify areas needing updates or clarification.

    Provide a detailed analysis of documentation accuracy.
    The file content is bellow, so no need to try to read it again.

    ### Current File Content:
    {doc_content}""",
    expected_output="""
    A comprehensive detailed analysis report covering:

    - Documentation completeness and accuracy
    - Alignment with current codebase
    - Identified discrepancies
    - Missing or outdated sections
    - Areas needing updates""",
    agent=code_analyzer
)

write_documentation = Task(
    description="""Based on the analysis, update the documentation:

    1. Update inaccurate or outdated content
    2. Add missing information and examples
    3. Ensure code examples match current implementation
    4. Maintain existing document structure
    5. Return complete updated markdown document

    Ensure all updates maintain technical accuracy while being clear and user-friendly.

    ### Original File Content:
    {doc_content}""",
    expected_output="""
    An updates markdown document that:

    - Accurately reflects current codebase
    - Maintains clear explanations
    - Includes up-to-date examples
    - Preserves document structure
    - Integrates all necessary updates
    - Don't start nor end with a code block
    """,
    agent=documentation_writer
)

In [6]:
# Create a flow to process documentation files
from pathlib import Path
from pydantic import BaseModel
from crewai.flow.flow import Flow, listen, start

class DocsState(BaseModel):
    files_path: list[str] = []

class DocumentationFlow(Flow[DocsState]):
    @start()
    def load_files_path(self):
        docs_dir = Path("/Users/joaomoura/workspace/nvidia/crewAI")
        print(f"Processing Docs: {docs_dir}")
        self.state.files_path = [str(file_path) for file_path in docs_dir.rglob("*.md")]

    @listen(load_files_path)
    def update_docs(self):
        print(f"Processing {self.state.files_path} files")
        print(f"Processing {len(self.state.files_path)} files")
        for file_path in self.state.files_path:
            print(f"Processing file: {file_path}")
            with open(file_path, 'r') as f:
                file_content = f.read()
            print("Running documentation crew...")
            documentation_crew = Crew(
                agents=[code_analyzer, documentation_writer],
                tasks=[analyze_code, write_documentation],
                verbose=True
            )
            # Use synchronous kickoff instead of async
            result = documentation_crew.kickoff(inputs={
              'file_path': file_path,
              'doc_content': file_content
            })

            content = result.raw
            with open(file_path, 'w') as f:
                f.write(content)

def kickoff():
    import nest_asyncio
    import asyncio

    # Apply a patch to allow nested asyncio loops in Jupyter
    nest_asyncio.apply()

    flow = DocumentationFlow()

    # Get or create an event loop and run the flow asynchronously
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(flow.kickoff_async())
    return result

def plot():
    flow = DocumentationFlow()
    flow.plot()

In [7]:
plot()

Plot saved as crewai_flow.html


In [8]:
# Display the flow visualization using IFrame
from IPython.display import IFrame

# Display the flow visualization
IFrame(src='./crewai_flow.html', width='100%', height=600)


In [9]:
kickoff()

Processing Docs: /Users/joaomoura/workspace/nvidia/crewAI
Processing ['/Users/joaomoura/workspace/nvidia/crewAI/README.md', '/Users/joaomoura/workspace/nvidia/crewAI/.github/security.md', '/Users/joaomoura/workspace/nvidia/crewAI/src/crewai/cli/templates/crew/README.md', '/Users/joaomoura/workspace/nvidia/crewAI/src/crewai/cli/templates/tool/README.md', '/Users/joaomoura/workspace/nvidia/crewAI/src/crewai/cli/templates/flow/README.md'] files
Processing 5 files
Processing file: /Users/joaomoura/workspace/nvidia/crewAI/README.md
Running documentation crew...
# Agent: Code Analyzer
## Task: Analyze the provided documentation file content:

    1. Read and understand the current documentation.
    2. Identify key concepts and components described.
    3. Compare documentation against actual codebase.
    4. Note any discrepancies or outdated information.
    5. Identify areas needing updates or clarification.

    Provide a detailed analysis of documentation accuracy.
    The file content 

2024-12-14 02:42:30,118 - 8071896128 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Documentation Writer
## Final Answer: 
To accurately reflect the current codebase while maintaining clarity and usability, the documentation needs to be updated to include the discrepancies identified. The "sequential" process and the "SerperDevTool" class should be explicitly mentioned in the documentation.

Here is the updated markdown document that:

- Accurately reflects the current codebase
- Maintains clear explanations
- Includes up-to-date examples
- Preserves document structure
- Integrates all necessary updates

```markdown
# Quickstart

Build your first AI agent with CrewAI in under 5 minutes.

## Build your first CrewAI Agent

Let's create a simple crew that will help us `research` and `report` on the `latest AI developments` for a given topic or subject.

Before we proceed, make sure you have `crewai` and `crewai-tools` installed.
If you haven't installed them yet, you can do so by following the [installation guide](/installation).

Follow the steps below to get

KeyboardInterrupt: 